In [1]:
# Math, image processing and other useful libraries
from __future__ import print_function, unicode_literals, absolute_import, division
import os

import pandas as pd
import numpy as np
import cv2
from collections import OrderedDict
import copy
import math
import pickle
from matplotlib.ticker import MaxNLocator
from itertools import combinations

# Image processing
from skimage.measure import regionprops
from skimage.filters import meijering, sato, frangi, hessian, threshold_otsu
from skimage.morphology import extrema, skeletonize
from skimage.transform import probabilistic_hough_line
from skimage.draw import disk, circle_perimeter
from scipy.ndimage import gaussian_filter, grey_closing
from scipy.spatial import distance_matrix
from skimage import data, restoration, util
from roipoly import RoiPoly
from matplotlib_scalebar.scalebar import ScaleBar
from biosppy.signals import tools
from biosppy.stats import pearson_correlation
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

# Plotting
import matplotlib.pyplot as plt
import matplotlib.cm as pltc
import matplotlib.colors as colors
import seaborn as sns

# Widgets
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual
from IPython.display import display

# Feature Extraction (.py files by Teresa Parreira)
# from CytoSkeletonPropsMorph import CytoSkeletonPropsMorph
# from CytoSkeletonRegionPropsInt import RegionPropsInt
# from FreqAnalysis import FreqAnalysis
# from GLCM import GLCM

# Graph
import sknw
import networkx as nx
from scipy.signal import argrelextrema

# 
from skan import Skeleton, summarize,draw
from skan.csr import skeleton_to_csgraph, sholl_analysis,make_degree_image
import scipy as sp
import scipy.sparse
from matplotlib.patches import Circle
from ImageFeatures import ImageFeatures
from Functions import FeaturesFromCentroid, cv2toski,pylsdtoski,polar_to_cartesian, truncate_colormap, plot_hist, plot_pie, remove_not1D, quantitative_analysis,hist_bin,hist_lim,create_separate_DFs,branch,graphAnalysis
from Importing import label_image,init_import
from PreProcessingCYTO import cytoskeleton_preprocessing, df_cytoskeleton_preprocessing
from PreProcessingNUCL import excludeborder, nuc_pre, df_nuclei_preprocessing, nuclei_segmentation
#from Processing import analyze_cell

#from fractal_dimension import fractal_dimension
#from fractal_analysis_fxns import boxcount,boxcount_grayscale,fractal_dimension,fractal_dimension_grayscale,fractal_dimension_grayscale_DBC

print('📚 All libraries successfully imported 📚')

utils.py (10): Please use `distance_transform_edt` from the `scipy.ndimage` namespace, the `scipy.ndimage.morphology` namespace is deprecated.
utils.py (10): Please use `binary_fill_holes` from the `scipy.ndimage` namespace, the `scipy.ndimage.morphology` namespace is deprecated.
utils.py (11): Please use `find_objects` from the `scipy.ndimage` namespace, the `scipy.ndimage.measurements` namespace is deprecated.


📚 All libraries successfully imported 📚


model2d.py (237): distutils Version classes are deprecated. Use packaging.version instead.
model2d.py (237): distutils Version classes are deprecated. Use packaging.version instead.
model3d.py (275): distutils Version classes are deprecated. Use packaging.version instead.
model3d.py (275): distutils Version classes are deprecated. Use packaging.version instead.


In [2]:
### DATASET #1 (2D)
#folder    = os.getcwd() + "\\Datasets\\Set 1-a-tubulin_Sofia"
#options   = ["RGB","CYTO_DECONV","NUCL_DECONV"]

### DATASET #2 (3D)
folder    = os.getcwd() + "\\Datasets\\Set 3D"
options   = ["3D"]

data = init_import(folder,options)
#data['NUCL_PRE'] = pd.read_pickle(os.getcwd() + ".//NUCL_PRE_12fev.pkl")

>>> [3D] added.


In [4]:
nucDF = pd.read_pickle(".//nucDF_4sl.pkl")  

In [5]:
nucDF

,Img Index,ID,DNF:Area,DNF:BB Area,DNF:Perimeter,DNF:Area convex,DNF:Centroid,DNF:Centroid Divergence,DNF:Equivalent Diameter,DNF:Extent,DNF:Major Axis Length,DNF:Minor Axis Length,DNF:Height,DNF:Euler Number,DNF:Eccentricity,DNF:Circularity,DNF:Roundness,DNF:Orientation,DNF:Solidity
0,11,2,15543,56875,0,25851,"(7.5508589075468056, 342.7660683265779, 841.04...",0.0,30.963169,0.273284,129.230831,2.22299,4,1,0,0,1.184985,0,0.601253
1,11,12,4265,13132,0,6520,"(6.7650644783118405, 387.8454865181712, 87.637...",0.0,20.120563,0.324779,71.280746,2.696283,3,1,0,0,1.068771,0,0.654141
2,11,15,11674,40290,0,18560,"(6.817800239849237, 482.2028439266747, 683.678...",0.0,28.145374,0.289749,109.376928,3.083312,4,1,0,0,1.242448,0,0.628987
3,11,16,10950,45675,0,18279,"(7.522283105022831, 551.5593607305937, 866.380...",0.0,27.551072,0.239737,118.960004,2.464875,4,1,0,0,0.985195,0,0.599048
4,11,35,16184,51480,0,25041,"(7.233131487889273, 606.8769154720712, 1102.11...",0.0,31.383093,0.314375,138.673776,3.047694,4,1,0,0,1.071537,0,0.6463
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1312,9,39,16609,36456,0,22116,"(7.335721596724667, 947.0231802035041, 844.894...",0.0,31.655435,0.45559,129.043514,3.037574,2,1,0,0,1.269934,0,0.750995
1313,9,43,16398,33276,0,21296,"(7.5781802658860835, 386.76588608366876, 281.7...",0.0,31.520814,0.492788,156.759243,2.204678,1,1,0,0,0.849639,0,0.770004
1314,9,44,7153,15150,0,9651,"(7.760799664476443, 386.18076331609115, 1030.4...",0.0,23.905429,0.472145,111.064635,1.89635,1,1,0,0,0.738324,0,0.741167
1315,9,47,5811,10092,0,6775,"(7.616417139907073, 560.0536912751678, 539.647...",0.0,22.30578,0.575803,98.400343,2.140702,1,1,0,0,0.764131,0,0.857712


In [6]:
nucDF['Label'] =[label_image(int(row['Img Index'])) for index,row in nucDF.iterrows()]